<a href="https://colab.research.google.com/github/roshantac/EVA4/blob/master/Assignment15/B/MaskPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!!nvidia-smi

['Sat May 16 18:50:39 2020       ',
 '+-----------------------------------------------------------------------------+',
 '| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |',
 '|-------------------------------+----------------------+----------------------+',
 '| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |',
 '| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |',
 '|===============================+======================+======================|',
 '|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |',
 '| N/A   47C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |',
 '+-------------------------------+----------------------+----------------------+',
 '                                                                               ',
 '+-----------------------------------------------------------------------------+',
 '| Processes:                          

In [2]:
!git clone https://github.com/roshantac/Unet.git

Cloning into 'Unet'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 44 (delta 16), reused 31 (delta 9), pack-reused 0
Unpacking objects: 100% (44/44), done.


In [0]:
!mkdir Unet/data

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [0]:
!mkdir temp
#!cp '/content/drive/My Drive/Eva_DataSet.zip' temp
!cp '/content/drive/My Drive/sample10.zip' temp

In [0]:
destDataDir = '/content/temp/'
path_to_zip_file = '/content/temp/sample10.zip'
import zipfile
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(destDataDir)

In [0]:
!mv '/content/temp/sample10k/depth' '/content/Unet/data/'
!mv '/content/temp/sample10k/fgbg' '/content/Unet/data/'
!mv '/content/temp/sample10k/mask' '/content/Unet/data/'

In [0]:
!rm -rf temp

In [1]:
%cd Unet/

/content/Unet


In [0]:
import argparse
import logging
import os
import sys

import numpy as np
import torch
import torch.nn as nn
from torch import optim
from tqdm import tqdm

from eval import eval_net
from unet import UNet

from torch.utils.tensorboard import SummaryWriter
from utils.dataset import BasicDataset
from torch.utils.data import DataLoader, random_split

In [0]:
dir_img = 'data/fgbg/'
dir_mask = 'data/mask/'
dir_checkpoint = 'checkpoints/'

def train_net(net,
              device,
              epochs=5,
              batch_size=1,
              lr=0.001,
              val_percent=0.2,
              save_cp=True,
              img_scale=0.5):

    dataset = BasicDataset(dir_img, dir_mask, img_scale)
    n_val = int(len(dataset) * val_percent)
    n_train = len(dataset) - n_val
    train, val = random_split(dataset, [n_train, n_val])
    train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
    val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True, drop_last=True)

    writer = SummaryWriter(comment=f'LR_{lr}_BS_{batch_size}_SCALE_{img_scale}')
    global_step = 0

    logging.info(f'''Starting training:
        Epochs:          {epochs}
        Batch size:      {batch_size}
        Learning rate:   {lr}
        Training size:   {n_train}
        Validation size: {n_val}
        Checkpoints:     {save_cp}
        Device:          {device.type}
        Images scaling:  {img_scale}
    ''')

    optimizer = optim.RMSprop(net.parameters(), lr=lr, weight_decay=1e-8, momentum=0.9)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min' if net.n_classes > 1 else 'max', patience=2)
    if net.n_classes > 1:
        criterion = nn.CrossEntropyLoss()
    else:
        criterion = nn.BCEWithLogitsLoss()

    for epoch in range(epochs):
        net.train()

        epoch_loss = 0
        with tqdm(total=n_train, desc=f'Epoch {epoch + 1}/{epochs}', unit='img') as pbar:
            for batch in train_loader:
                imgs = batch['image']
                true_masks = batch['mask']
                assert imgs.shape[1] == net.n_channels, \
                    f'Network has been defined with {net.n_channels} input channels, ' \
                    f'but loaded images have {imgs.shape[1]} channels. Please check that ' \
                    'the images are loaded correctly.'

                imgs = imgs.to(device=device, dtype=torch.float32)
                mask_type = torch.float32 if net.n_classes == 1 else torch.long
                true_masks = true_masks.to(device=device, dtype=mask_type)

                masks_pred = net(imgs)
                loss = criterion(masks_pred, true_masks)
                epoch_loss += loss.item()
                writer.add_scalar('Loss/train', loss.item(), global_step)

                pbar.set_postfix(**{'loss (batch)': loss.item()})

                optimizer.zero_grad()
                loss.backward()
                nn.utils.clip_grad_value_(net.parameters(), 0.1)
                optimizer.step()

                pbar.update(imgs.shape[0])
                global_step += 1
                # if global_step % (len(dataset) // (10 * batch_size)) == 0:
                #     for tag, value in net.named_parameters():
                #         tag = tag.replace('.', '/')
                #         writer.add_histogram('weights/' + tag, value.data.cpu().numpy(), global_step)
                #         writer.add_histogram('grads/' + tag, value.grad.data.cpu().numpy(), global_step)
                #     val_score = eval_net(net, val_loader, device)
                #     scheduler.step(val_score)
                #     writer.add_scalar('learning_rate', optimizer.param_groups[0]['lr'], global_step)

                #     if net.n_classes > 1:
                #         logging.info('Validation cross entropy: {}'.format(val_score))
                #         writer.add_scalar('Loss/test', val_score, global_step)
                #     else:
                #         logging.info('Validation Dice Coeff: {}'.format(val_score))
                #         writer.add_scalar('Dice/test', val_score, global_step)

                #     writer.add_images('images', imgs, global_step)
                #     if net.n_classes == 1:
                #         writer.add_images('masks/true', true_masks, global_step)
                #         writer.add_images('masks/pred', torch.sigmoid(masks_pred) > 0.5, global_step)

        if save_cp:
            try:
                os.mkdir(dir_checkpoint)
                logging.info('Created checkpoint directory')
            except OSError:
                pass
            torch.save(net.state_dict(),
                       dir_checkpoint + f'CP_epoch{epoch + 1}.pth')
            logging.info(f'Checkpoint {epoch + 1} saved !')

    writer.close()



In [0]:
net = UNet(n_channels=3, n_classes=1, bilinear=True)

In [5]:
from torchsummary import summary
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net.to(device=device)
summary(net, (3,200,200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 200, 200]             896
       BatchNorm2d-2         [-1, 32, 200, 200]              64
              ReLU-3         [-1, 32, 200, 200]               0
            Conv2d-4         [-1, 32, 200, 200]           9,248
       BatchNorm2d-5         [-1, 32, 200, 200]              64
              ReLU-6         [-1, 32, 200, 200]               0
        DoubleConv-7         [-1, 32, 200, 200]               0
         MaxPool2d-8         [-1, 32, 100, 100]               0
            Conv2d-9         [-1, 64, 100, 100]          18,496
      BatchNorm2d-10         [-1, 64, 100, 100]             128
             ReLU-11         [-1, 64, 100, 100]               0
           Conv2d-12         [-1, 64, 100, 100]          36,928
      BatchNorm2d-13         [-1, 64, 100, 100]             128
             ReLU-14         [-1, 64, 1

In [6]:

logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

logging.info(f'Using device {device}')

# Change here to adapt to your data
# n_channels=3 for RGB images
# n_classes is the number of probabilities you want to get per pixel
#   - For 1 class and background, use n_classes=1
#   - For 2 classes, use n_classes=1
#   - For N > 2 classes, use n_classes=N

logging.info(f'Network:\n'
              f'\t{net.n_channels} input channels\n'
              f'\t{net.n_classes} output channels (classes)\n'
              f'\t{"Bilinear" if net.bilinear else "Transposed conv"} upscaling')


# faster convolutions, but more memorycudnn.benchmark = True
train_net(net=net,
          epochs=10,
          batch_size=32,
          lr=0.01,
          device=device,
          img_scale=1.0,
          val_percent= 10.0/ 100)
# except KeyboardInterrupt:
#     torch.save(net.state_dict(), 'INTERRUPTED.pth')
#     logging.info('Saved interrupt')
#     try:
#         sys.exit(0)
#     except SystemExit:
#         os._exit(0)


INFO: Using device cuda
INFO: Network:
	3 input channels
	1 output channels (classes)
	Bilinear upscaling
INFO: Starting training:
        Epochs:          10
        Batch size:      32
        Learning rate:   0.01
        Training size:   9000
        Validation size: 1000
        Checkpoints:     True
        Device:          cuda
        Images scaling:  1.0
    
Epoch 1/10: 100%|██████████| 9000/9000 [01:51<00:00, 80.37img/s, loss (batch)=0.0619]
INFO: Created checkpoint directory
INFO: Checkpoint 1 saved !
Epoch 2/10: 100%|██████████| 9000/9000 [01:53<00:00, 79.59img/s, loss (batch)=0.049]
INFO: Checkpoint 2 saved !
Epoch 3/10: 100%|██████████| 9000/9000 [01:52<00:00, 79.68img/s, loss (batch)=0.0246]
INFO: Checkpoint 3 saved !
Epoch 4/10: 100%|██████████| 9000/9000 [01:53<00:00, 79.46img/s, loss (batch)=0.0282]
INFO: Checkpoint 4 saved !
Epoch 5/10: 100%|██████████| 9000/9000 [01:52<00:00, 79.75img/s, loss (batch)=0.0334]
INFO: Checkpoint 5 saved !
Epoch 6/10: 100%|██████████| 9

In [0]:
#!cp checkpoints/*.pth '/content/drive/My Drive/Untitled folder/'

In [0]:
#!mkdir sample50
#!mkdir sample50/out
!mkdir sample50/ind
from predict import *
from PIL import Image
sdir = 'sample50/out/'
indir ='sample50/ind/'
for i,idx in enumerate(os.listdir('data/fgbg/')):
  if i==100:
    break;
  img = Image.open('data/fgbg/'+idx)
  img.save(indir+idx)
  mask = predict_img(net=net,
                      full_img=img,
                      scale_factor=1.0,
                      out_threshold=0.2,
                      device=device)
  result = mask_to_image(mask)
  result.save(sdir+idx)


In [0]:
mkdir sample50/out

In [0]:
from os import listdir
from PIL import Image 
from predict import predict_img
from matplotlib import cm
import matplotlib
net = UNet(3, 1).cuda()
dire = 'sample50/fgbg/'
sdir = 'sample50/out/'
sdir = ''
lst_msk = []
net.load_state_dict(torch.load('/content/drive/My Drive/models/CP_epoch1.pth'))
for i,dx in enumerate(listdir(dire)):
  img = Image.open(dire+dx)
  mask = predict_img(net, img, device)
  matplotlib.image.imsave(sdir+dx, mask)
  #im.save(sdir+dx)
  lst_msk.append(mask)


In [30]:
len(lst_msk)

50

In [0]:
import shutil
# !mkdir sample10k
# !mkdir sample10k/fgbg
# !mkdir sample10k/depth
# !mkdir sample10k/mask
# !mkdir sample50
# !mkdir sample50/fgbg
# !mkdir sample50/depth
# !mkdir sample50/mask
rootsrc = 'data/'
import os
for i,idx in enumerate(os.listdir(rootsrc+'fgbg/')):
  if( i == 10000):
    break
  shutil.copy(rootsrc + 'fgbg/' + idx, 'sample10k/fgbg/') 
  shutil.copy(rootsrc + 'fgbgmask/' + idx, 'sample10k/mask/') 
  shutil.copy(rootsrc + 'Depthmap/' + idx, 'sample10k/depth/') 
  if (i<50):
    shutil.copy(rootsrc + 'fgbg/' + idx, 'sample50/fgbg/') 
    shutil.copy(rootsrc + 'fgbgmask/' + idx, 'sample50/mask/') 
    shutil.copy(rootsrc + 'Depthmap/' + idx, 'sample50/depth/') 

In [0]:
!cp -r sample50 '/content/drive/My Drive/'

In [17]:
from zipfile import ZipFile 
import os 
  
def get_all_file_paths(directory): 
  
    # initializing empty file paths list 
    file_paths = [] 
  
    # crawling through directory and subdirectories 
    for root, directories, files in os.walk(directory): 
        for filename in files: 
            # join the two strings in order to form the full filepath. 
            filepath = os.path.join(root, filename) 
            file_paths.append(filepath) 
  
    # returning all file paths 
    return file_paths         
  
def main(): 
    # path to folder which needs to be zipped 
    directory = 'sample10k/'
  
    # calling function to get all file paths in the directory 
    file_paths = get_all_file_paths(directory) 
  
    # printing the list of all files to be zipped 
    print('Following files will be zipped:') 
    for file_name in file_paths: 
        print(file_name) 
  
    # writing files to a zipfile 
    with ZipFile('sample10.zip','w') as zip: 
        # writing each file one by one 
        for file in file_paths: 
            zip.write(file) 
  
    print('All files zipped successfully!')  
main()

Streaming output truncated to the last 5000 lines.
sample10k/fgbg/img392297.jpg
sample10k/fgbg/img37764.jpg
sample10k/fgbg/img62657.jpg
sample10k/fgbg/img356466.jpg
sample10k/fgbg/img246844.jpg
sample10k/fgbg/img185518.jpg
sample10k/fgbg/img308780.jpg
sample10k/fgbg/img78359.jpg
sample10k/fgbg/img261146.jpg
sample10k/fgbg/img19220.jpg
sample10k/fgbg/img369324.jpg
sample10k/fgbg/img263610.jpg
sample10k/fgbg/img183121.jpg
sample10k/fgbg/img339848.jpg
sample10k/fgbg/img150959.jpg
sample10k/fgbg/img161126.jpg
sample10k/fgbg/img53166.jpg
sample10k/fgbg/img348738.jpg
sample10k/fgbg/img208940.jpg
sample10k/fgbg/img184839.jpg
sample10k/fgbg/img176042.jpg
sample10k/fgbg/img148918.jpg
sample10k/fgbg/img34667.jpg
sample10k/fgbg/img333899.jpg
sample10k/fgbg/img406873.jpg
sample10k/fgbg/img181900.jpg
sample10k/fgbg/img125640.jpg
sample10k/fgbg/img176018.jpg
sample10k/fgbg/img375365.jpg
sample10k/fgbg/img45140.jpg
sample10k/fgbg/img387061.jpg
sample10k/fgbg/img95663.jpg
sample10k/fgbg/img215811.jpg


In [0]:
!cp sample10.zip '/content/drive/My Drive/'